# 제품 이상여부 판별 프로젝트


## 1. 데이터 불러오기


### 필수 라이브러리


In [1]:
import os
from pprint import pprint

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

### 데이터 읽어오기


In [2]:
ROOT_DIR = "data"

# Load data
train_data = pd.read_csv(os.path.join(ROOT_DIR, "train.csv"))
train_data

,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION X Unit Time_Dam,CURE END POSITION X Judge Value_Dam,...,Production Qty Collect Result_Fill2,Production Qty Unit Time_Fill2,Production Qty Judge Value_Fill2,Receip No Collect Result_Fill2,Receip No Unit Time_Fill2,Receip No Judge Value_Fill2,WorkMode Collect Result_Fill2,WorkMode Unit Time_Fill2,WorkMode Judge Value_Fill2,target
0,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XA938-1,1,OK,240.0,NaN,NaN,...,7,NaN,NaN,127,NaN,NaN,1,NaN,NaN,Normal
1,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,3KPM0016-2,1,OK,240.0,NaN,NaN,...,185,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
2,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1X9167-1,1,OK,1000.0,NaN,NaN,...,10,NaN,NaN,73,NaN,NaN,1,NaN,NaN,Normal
3,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1X0057-1,1,OK,1000.0,NaN,NaN,...,268,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
4,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3HPM0007-1,1,OK,240.0,NaN,NaN,...,121,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3J1XF434-2,1,OK,240.0,NaN,NaN,...,318,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
40502,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1XC796-1,1,OK,1000.0,NaN,NaN,...,14,NaN,NaN,197,NaN,NaN,1,NaN,NaN,Normal
40503,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4C1XD438-1,1,OK,240.0,NaN,NaN,...,1,NaN,NaN,27,NaN,NaN,1,NaN,NaN,Normal
40504,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3I1XA258-1,1,OK,1000.0,NaN,NaN,...,117,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal


In [3]:
# Drop columns with more than half of the values missing
drop_cols = []
for column in train_data.columns:
    if (train_data[column].notnull().sum() // 2) < train_data[
        column
    ].isnull().sum():
        drop_cols.append(column)
train_data = train_data.drop(drop_cols, axis=1)

train_data

,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,...,Head Clean Position Z Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2,target
0,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XA938-1,1,OK,240.0,2.5,-90,...,50.0,91.8,270,50,114.612,19.9,7,127,1,Normal
1,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,3KPM0016-2,1,OK,240.0,2.5,-90,...,91.8,270.0,50,85,19.600,7.0,185,1,0,Normal
2,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1X9167-1,1,OK,1000.0,12.5,90,...,50.0,91.8,270,50,114.612,19.8,10,73,1,Normal
3,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1X0057-1,1,OK,1000.0,12.5,90,...,91.8,270.0,50,85,19.900,12.0,268,1,0,Normal
4,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3HPM0007-1,1,OK,240.0,2.5,-90,...,91.8,270.0,50,85,19.700,8.0,121,1,0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3J1XF434-2,1,OK,240.0,2.5,-90,...,91.8,270.0,50,85,19.200,1.0,318,1,0,Normal
40502,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1XC796-1,1,OK,1000.0,12.5,90,...,50.0,91.8,270,50,114.612,20.5,14,197,1,Normal
40503,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4C1XD438-1,1,OK,240.0,2.5,-90,...,50.0,91.8,270,50,85.000,19.7,1,27,1,Normal
40504,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3I1XA258-1,1,OK,1000.0,12.5,90,...,91.8,270.0,50,85,20.100,13.0,117,1,0,Normal


In [4]:
# 각 열의 NaN 값 개수 계산
nan_counts = train_data.isnull().sum()

# NaN 값이 있는 열들만 필터링
nan_columns = nan_counts[nan_counts > 0]

nan_columns

HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam      12766
HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1    12766
HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2    12766
dtype: int64

In [5]:
# 각 열의 NaN 값이 있는 행을 표시
nan_dam = train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam'].isnull()
nan_fill1 = train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1'].isnull()
nan_fill2 = train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'].isnull()

# 세 열의 NaN 값이 모두 같은 행에서 발생했는지 확인
nan_same_rows = nan_dam & nan_fill1 & nan_fill2

# NaN 값이 동일한 행의 개수 확인
num_same_nan_rows = nan_same_rows.sum()

# 결과 출력
print(f"세 열에서 NaN 값이 동일한 행의 개수: {num_same_nan_rows}")
print(f"세 열에서 NaN 값이 동일한 행:\n{train_data[nan_same_rows]}")

세 열에서 NaN 값이 동일한 행의 개수: 12766
세 열에서 NaN 값이 동일한 행:
      Wip Line_Dam Process Desc._Dam     Equipment_Dam Model.Suffix_Dam  \
0          IVI-OB6     Dam Dispenser  Dam dispenser #1      AJX75334505   
12         IVI-OB6     Dam Dispenser  Dam dispenser #2      AJX75334501   
13         IVI-OB6     Dam Dispenser  Dam dispenser #2      AJX75334501   
18         IVI-OB6     Dam Dispenser  Dam dispenser #1      AJX75334501   
24         IVI-OB6     Dam Dispenser  Dam dispenser #1      AJX75334507   
...            ...               ...               ...              ...   
40485      IVI-OB6     Dam Dispenser  Dam dispenser #1      AJX75334501   
40491      IVI-OB6     Dam Dispenser  Dam dispenser #2      AJX75334501   
40492      IVI-OB6     Dam Dispenser  Dam dispenser #2      AJX75334501   
40500      IVI-OB6     Dam Dispenser  Dam dispenser #2      AJX75334501   
40502      IVI-OB6     Dam Dispenser  Dam dispenser #2      AJX75334501   

       Workorder_Dam  Insp. Seq No._Dam Insp Judg

In [6]:
# 열 목록을 정의
required_columns = [
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Dam',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill1',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill2',
    'GMES_ORIGIN_INSP_JUDGE_CODE Collect Result_AutoClave',
    'GMES_ORIGIN_INSP_JUDGE_CODE Judge Value_AutoClave'
]

# 존재하는 열들만 필터링
existing_columns = [col for col in required_columns if col in train_data.columns]

# 존재하는 열들 출력
print("존재하는 열들:", existing_columns)

존재하는 열들: []


In [7]:
# Model.Suffix의 값이 공정에 따라 같은지 다른지 비교할 열 리스트
suffix_columns = [
    'Model.Suffix_Dam',
    'Model.Suffix_Fill1',
    'Model.Suffix_Fill2',
    'Model.Suffix_AutoClave'
]

# 각 행에 대해 모든 비교 열의 값이 동일한지 확인
all_equal = train_data[suffix_columns].nunique(axis=1) == 1

# 동일한 값을 가진 행만 필터링하여 네 개의 열만 선택
equal_rows = train_data.loc[all_equal, suffix_columns]

# 동일한 값을 가진 행의 개수와 결과 출력
print(f"동일한 값을 가진 행의 개수: {len(equal_rows)}")
print(equal_rows)

동일한 값을 가진 행의 개수: 40506
      Model.Suffix_Dam Model.Suffix_Fill1 Model.Suffix_Fill2  \
0          AJX75334505        AJX75334505        AJX75334505   
1          AJX75334505        AJX75334505        AJX75334505   
2          AJX75334501        AJX75334501        AJX75334501   
3          AJX75334501        AJX75334501        AJX75334501   
4          AJX75334501        AJX75334501        AJX75334501   
...                ...                ...                ...   
40501      AJX75334501        AJX75334501        AJX75334501   
40502      AJX75334501        AJX75334501        AJX75334501   
40503      AJX75334501        AJX75334501        AJX75334501   
40504      AJX75334501        AJX75334501        AJX75334501   
40505      AJX75334501        AJX75334501        AJX75334501   

      Model.Suffix_AutoClave  
0                AJX75334505  
1                AJX75334505  
2                AJX75334501  
3                AJX75334501  
4                AJX75334501  
...                   

In [8]:
# Workorder의 값이 공정에 따라 같은지 다른지 비교할 열 리스트
suffix_columns = [
    'Workorder_Dam',
    'Workorder_Fill1',
    'Workorder_Fill2',
    'Workorder_AutoClave'
]

# 각 행에 대해 모든 비교 열의 값이 동일한지 확인
all_equal = train_data[suffix_columns].nunique(axis=1) == 1

# 동일한 값을 가진 행만 필터링하여 네 개의 열만 선택
equal_rows = train_data.loc[all_equal, suffix_columns]

# 동일한 값을 가진 행의 개수와 결과 출력
print(f"동일한 값을 가진 행의 개수: {len(equal_rows)}")
print(equal_rows)

동일한 값을 가진 행의 개수: 40506
      Workorder_Dam Workorder_Fill1 Workorder_Fill2 Workorder_AutoClave
0        4F1XA938-1      4F1XA938-1      4F1XA938-1          4F1XA938-1
1        3KPM0016-2      3KPM0016-2      3KPM0016-2          3KPM0016-2
2        4E1X9167-1      4E1X9167-1      4E1X9167-1          4E1X9167-1
3        3K1X0057-1      3K1X0057-1      3K1X0057-1          3K1X0057-1
4        3HPM0007-1      3HPM0007-1      3HPM0007-1          3HPM0007-1
...             ...             ...             ...                 ...
40501    3J1XF434-2      3J1XF434-2      3J1XF434-2          3J1XF434-2
40502    4E1XC796-1      4E1XC796-1      4E1XC796-1          4E1XC796-1
40503    4C1XD438-1      4C1XD438-1      4C1XD438-1          4C1XD438-1
40504    3I1XA258-1      3I1XA258-1      3I1XA258-1          3I1XA258-1
40505    3G1XA501-1      3G1XA501-1      3G1XA501-1          3G1XA501-1

[40506 rows x 4 columns]


In [9]:
# 삭제할 열 리스트
columns_to_drop = [
        'Model.Suffix_Fill1', 'Model.Suffix_Fill2', 'Model.Suffix_AutoClave', 
        'Workorder_Fill1', 'Workorder_Fill2', 'Workorder_AutoClave'
]

# 열 삭제
train_data = train_data.drop(columns=columns_to_drop)

# 열 이름 변경
train_data = train_data.rename(columns={
    'Model.Suffix_Dam': 'Model.Suffix',
    'Workorder_Dam': 'Workorder'
})

train_data

,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix,Workorder,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,...,Head Clean Position Z Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2,target
0,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XA938-1,1,OK,240.0,2.5,-90,...,50.0,91.8,270,50,114.612,19.9,7,127,1,Normal
1,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,3KPM0016-2,1,OK,240.0,2.5,-90,...,91.8,270.0,50,85,19.600,7.0,185,1,0,Normal
2,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1X9167-1,1,OK,1000.0,12.5,90,...,50.0,91.8,270,50,114.612,19.8,10,73,1,Normal
3,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1X0057-1,1,OK,1000.0,12.5,90,...,91.8,270.0,50,85,19.900,12.0,268,1,0,Normal
4,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3HPM0007-1,1,OK,240.0,2.5,-90,...,91.8,270.0,50,85,19.700,8.0,121,1,0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3J1XF434-2,1,OK,240.0,2.5,-90,...,91.8,270.0,50,85,19.200,1.0,318,1,0,Normal
40502,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1XC796-1,1,OK,1000.0,12.5,90,...,50.0,91.8,270,50,114.612,20.5,14,197,1,Normal
40503,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4C1XD438-1,1,OK,240.0,2.5,-90,...,50.0,91.8,270,50,85.000,19.7,1,27,1,Normal
40504,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3I1XA258-1,1,OK,1000.0,12.5,90,...,91.8,270.0,50,85,20.100,13.0,117,1,0,Normal


In [10]:
# 모든 행이 동일한 값을 가지는 열을 찾고, 그 값을 함께 저장
constant_columns = {col: train_data[col].iloc[0] for col in train_data.columns if train_data[col].nunique() == 1}

# 데이터 프레임에서 해당 열 삭제
train_data = train_data.drop(columns=constant_columns.keys())

# 삭제된 열의 개수 출력
print(f"삭제된 열의 개수: {len(constant_columns)}")

# 삭제된 열과 그 값을 한 줄에 하나씩 출력
print("삭제된 열과 값:")
for col, value in constant_columns.items():
    print(f"{col}: {value}")

train_data

삭제된 열의 개수: 35
삭제된 열과 값:
Wip Line_Dam: IVI-OB6
Process Desc._Dam: Dam Dispenser
Insp. Seq No._Dam: 1
Insp Judge Code_Dam: OK
CURE STANDBY POSITION X Collect Result_Dam: 1150
CURE STANDBY POSITION Z Collect Result_Dam: 33.5
CURE STANDBY POSITION Θ Collect Result_Dam: 0
CURE START POSITION Z Collect Result_Dam: 33.5
Wip Line_AutoClave: IVI-OB6
Process Desc._AutoClave: Auto Clave Out
Equipment_AutoClave: Auto Clave Out
Insp. Seq No._AutoClave: 1
Insp Judge Code_AutoClave: OK
1st Pressure Judge Value_AutoClave: OK
2nd Pressure Judge Value_AutoClave: OK
3rd Pressure Judge Value_AutoClave: OK
Wip Line_Fill1: IVI-OB6
Process Desc._Fill1: Fill1 Dispenser
Insp. Seq No._Fill1: 1
Insp Judge Code_Fill1: OK
Wip Line_Fill2: IVI-OB6
Process Desc._Fill2: Fill2 Dispenser
Insp. Seq No._Fill2: 1
Insp Judge Code_Fill2: OK
CURE END POSITION Θ Collect Result_Fill2: -90
CURE STANDBY POSITION X Collect Result_Fill2: 1020
CURE STANDBY POSITION Θ Collect Result_Fill2: 0
CURE START POSITION Θ Collect Result_Fill2

,Equipment_Dam,Model.Suffix,Workorder,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE START POSITION X Collect Result_Dam,CURE START POSITION Θ Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,...,Head Clean Position Z Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2,target
0,Dam dispenser #1,AJX75334505,4F1XA938-1,240.0,2.5,-90,100,1030,-90,16,...,50.0,91.8,270,50,114.612,19.9,7,127,1,Normal
1,Dam dispenser #1,AJX75334505,3KPM0016-2,240.0,2.5,-90,70,1030,-90,10,...,91.8,270.0,50,85,19.600,7.0,185,1,0,Normal
2,Dam dispenser #2,AJX75334501,4E1X9167-1,1000.0,12.5,90,85,280,90,16,...,50.0,91.8,270,50,114.612,19.8,10,73,1,Normal
3,Dam dispenser #2,AJX75334501,3K1X0057-1,1000.0,12.5,90,70,280,90,10,...,91.8,270.0,50,85,19.900,12.0,268,1,0,Normal
4,Dam dispenser #1,AJX75334501,3HPM0007-1,240.0,2.5,-90,70,1030,-90,10,...,91.8,270.0,50,85,19.700,8.0,121,1,0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,Dam dispenser #1,AJX75334501,3J1XF434-2,240.0,2.5,-90,70,1030,-90,10,...,91.8,270.0,50,85,19.200,1.0,318,1,0,Normal
40502,Dam dispenser #2,AJX75334501,4E1XC796-1,1000.0,12.5,90,100,280,90,16,...,50.0,91.8,270,50,114.612,20.5,14,197,1,Normal
40503,Dam dispenser #1,AJX75334501,4C1XD438-1,240.0,2.5,-90,100,1030,-90,16,...,50.0,91.8,270,50,85.000,19.7,1,27,1,Normal
40504,Dam dispenser #2,AJX75334501,3I1XA258-1,1000.0,12.5,90,70,280,90,10,...,91.8,270.0,50,85,20.100,13.0,117,1,0,Normal


In [11]:
# 열 이름 정의
target_column = 'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam'

# 열의 값들 확인
if target_column not in train_data.columns:
    raise ValueError(f"열 '{target_column}'이(가) 데이터프레임에 없습니다.")

# 고유한 값 추출
unique_values = train_data[target_column].unique()

# 고유한 값들을 포함한 DataFrame 생성
unique_values_df = pd.DataFrame(unique_values, columns=[target_column])

# 결측치와 문자열 포함 모든 고유 값 출력
print(f"'{target_column}' 열의 고유한 값들:")
print(unique_values_df)

# 추가로 결측치와 특정 값('OK')의 존재 여부 확인
nan_count = train_data[target_column].isna().sum()
ok_count = (train_data[target_column] == 'OK').sum()

print(f"\n결측치(NaN) 개수: {nan_count}")
print(f"'OK' 값 개수: {ok_count}")

'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam' 열의 고유한 값들:
  HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam
0                                                NaN      
1                                              550.3      
2                                                 OK      
3                                              162.4      
4                                                549      
5                                              549.5      
6                                                550      
7                                              548.5      

결측치(NaN) 개수: 12766
'OK' 값 개수: 11293


In [12]:
data = train_data.copy()

In [13]:
# # 제거할 열 리스트
columns_to_drop = ['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam', 
                   'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1', 
                   'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2']
#   'WorkMode Collect Result_Dam', 'WorkMode Collect Result_Fill1', 'WorkMode Collect Result_Fill2'
# # 지정한 열 제거
data = data.drop(columns=columns_to_drop)

In [14]:
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline

# 범주형 열을 선택하여 인코딩 수행
categorical_features = ['Equipment_Dam', 'Equipment_Fill1', 'Equipment_Fill2', 'Chamber Temp. Judge Value_AutoClave', 'Model.Suffix', 'Workorder']

# OrdinalEncoder 인스턴스 생성
encoder = OrdinalEncoder()

# 범주형 열에 대해서만 인코딩 수행
data[categorical_features] = encoder.fit_transform(data[categorical_features])

# 인코딩된 데이터프레임 확인
data

,Equipment_Dam,Model.Suffix,Workorder,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE START POSITION X Collect Result_Dam,CURE START POSITION Θ Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,...,Head Clean Position Z Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2,target
0,0.0,3.0,657.0,240.0,2.5,-90,100,1030,-90,16,...,50.0,91.8,270,50,114.612,19.9,7,127,1,Normal
1,0.0,3.0,283.0,240.0,2.5,-90,70,1030,-90,10,...,91.8,270.0,50,85,19.600,7.0,185,1,0,Normal
2,1.0,0.0,589.0,1000.0,12.5,90,85,280,90,16,...,50.0,91.8,270,50,114.612,19.8,10,73,1,Normal
3,1.0,0.0,251.0,1000.0,12.5,90,70,280,90,10,...,91.8,270.0,50,85,19.900,12.0,268,1,0,Normal
4,0.0,0.0,142.0,240.0,2.5,-90,70,1030,-90,10,...,91.8,270.0,50,85,19.700,8.0,121,1,0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,0.0,0.0,238.0,240.0,2.5,-90,70,1030,-90,10,...,91.8,270.0,50,85,19.200,1.0,318,1,0,Normal
40502,1.0,0.0,643.0,1000.0,12.5,90,100,280,90,16,...,50.0,91.8,270,50,114.612,20.5,14,197,1,Normal
40503,0.0,0.0,540.0,240.0,2.5,-90,100,1030,-90,16,...,50.0,91.8,270,50,85.000,19.7,1,27,1,Normal
40504,1.0,0.0,164.0,1000.0,12.5,90,70,280,90,10,...,91.8,270.0,50,85,20.100,13.0,117,1,0,Normal


In [15]:
# 각 범주에 대한 인코딩 값 확인
for feature, categories in zip(categorical_features, encoder.categories_):
    print(f"Feature: {feature}")
    for category, index in zip(categories, range(len(categories))):
        print(f"  {category}: {index}")

Feature: Equipment_Dam
  Dam dispenser #1: 0
  Dam dispenser #2: 1
Feature: Equipment_Fill1
  Fill1 dispenser #1: 0
  Fill1 dispenser #2: 1
Feature: Equipment_Fill2
  Fill2 dispenser #1: 0
  Fill2 dispenser #2: 1
Feature: Chamber Temp. Judge Value_AutoClave
  NG: 0
  OK: 1
Feature: Model.Suffix
  AJX75334501: 0
  AJX75334502: 1
  AJX75334503: 2
  AJX75334505: 3
  AJX75334506: 4
  AJX75334507: 5
  AJX75334508: 6
Feature: Workorder
  3F1X5847-2: 0
  3F1X9643-1: 1
  3F1X9644-1: 2
  3F1X9648-1: 3
  3F1X9648-2: 4
  3F1XA350-1: 5
  3F1XA351-1: 6
  3F1XB560-1: 7
  3F1XC376-1: 8
  3F1XC414-1: 9
  3F1XC414-2: 10
  3F1XC600-1: 11
  3F1XC781-1: 12
  3FPM0081-1: 13
  3FPM0085-1: 14
  3FPXX064-0003: 15
  3G1X4501-1: 16
  3G1X4502-2: 17
  3G1X4503-1: 18
  3G1X8290-1: 19
  3G1X8293-1: 20
  3G1X8295-1: 21
  3G1X8296-1: 22
  3G1X8297-1: 23
  3G1X8297-2: 24
  3G1X8298-1: 25
  3G1X8298-2: 26
  3G1X8299-1: 27
  3G1X8300-1: 28
  3G1X8300-2: 29
  3G1X8303-1: 30
  3G1X8646-1: 31
  3G1X8646-2: 32
  3G1X8647-1

In [16]:
# 문자열 타입인 열들의 이름과 데이터 타입 출력
string_columns = data.select_dtypes(include='object')
string_columns.dtypes

target    object
dtype: object

##  train/val split,  df_abnormal 구분, 언더샘플링,  장비 구분

In [17]:
categorical_features = ['Equipment_Dam', 'Equipment_Fill1', 'Equipment_Fill2']
# 세 가지 조건: 열 간 값이 다르면 불량으로 분류
condition_receip = (
    (data['Receip No Collect Result_Dam'] != data['Receip No Collect Result_Fill1']) |
    (data['Receip No Collect Result_Dam'] != data['Receip No Collect Result_Fill2']) |
    (data['Receip No Collect Result_Fill1'] != data['Receip No Collect Result_Fill2'])
)

condition_production_qty = (
    (data['Production Qty Collect Result_Dam'] != data['Production Qty Collect Result_Fill1']) |
    (data['Production Qty Collect Result_Dam'] != data['Production Qty Collect Result_Fill2']) |
    (data['Production Qty Collect Result_Fill1'] != data['Production Qty Collect Result_Fill2'])
)

# 장비가 다르면 불량으로 분류
equip1 = data[categorical_features].eq(0).all(axis=1)
equip2 = data[categorical_features].eq(1).all(axis=1)
condition_equip_different = ~equip1 & ~equip2

# 네 조건 중 하나라도 만족하면 불량으로 분류
condition_abnormal = condition_receip | condition_production_qty | condition_equip_different

# 불량 데이터와 정상 데이터로 분리
df_abnormal = data[condition_abnormal]  # 불량으로 분류할 데이터
df_filtered = data[~condition_abnormal]  # 모델1, 2로 학습할 데이터


# 결과 확인
print(f"불량 데이터: {len(df_abnormal)} 행")
print(f"정상 데이터: {len(df_filtered)} 행")

불량 데이터: 93 행
정상 데이터: 40413 행


In [18]:
df_abnormal['target'].value_counts()

target
AbNormal    93
Name: count, dtype: int64

In [19]:
print(data.shape)
print(df_filtered.shape)

(40506, 137)
(40413, 137)


In [20]:
RANDOM_STATE = 42

normal_ratio = 4.0  # 1.0 means 1:1 ratio

df_normal = df_filtered[df_filtered["target"] == "Normal"]
df_abnormal = df_filtered[df_filtered["target"] == "AbNormal"]

num_normal = len(df_normal)
num_abnormal = len(df_abnormal)
print(f"  Total: Normal: {num_normal}, AbNormal: {num_abnormal}")

df_normal = df_normal.sample(n=int(num_abnormal * normal_ratio), replace=False, random_state=RANDOM_STATE)
df_concat = pd.concat([df_normal, df_abnormal], axis=0).reset_index(drop=True)
df_concat.value_counts("target")

  Total: Normal: 38156, AbNormal: 2257


target
Normal      9028
AbNormal    2257
Name: count, dtype: int64

In [21]:
# 정상 데이터에서 장비별로 데이터 분리
df_model1 = df_concat[df_concat[categorical_features].eq(0).all(axis=1)]
df_model2 = df_concat[df_concat[categorical_features].eq(1).all(axis=1)]

print(f"Equipment #1 데이터: {len(df_model1)} 행")
print(f"Equipment #2 데이터: {len(df_model2)} 행")

Equipment #1 데이터: 7036 행
Equipment #2 데이터: 4249 행


In [22]:
# 데이터와 타겟 분리
X_model1 = df_model1.drop(columns='target')
y_model1 = df_model1['target']
X_model2 = df_model2.drop(columns='target')
y_model2 = df_model2['target']

##  RandomForest

In [23]:
RANDOM_STATE = 42

# 모델 훈련
model1 = RandomForestClassifier(random_state=42)
model1.fit(X_model1, y_model1)

model2 = RandomForestClassifier(random_state=42)
model2.fit(X_model2, y_model2)

RandomForestClassifier(random_state=42)

In [24]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': [100, 200, 500],  # 적절한 트리 수
    'max_depth': [30, 50, None],  # 깊이를 넓게 설정
    'min_samples_split': [10, 20],  # 분할을 더 자유롭게 설정
    'min_samples_leaf': [1, 2, 4, 8, 16],  # 리프 노드 최소 샘플 수
    'criterion': ['gini']
    #'max_features': ['auto', 'sqrt', 'log2']  # 다양한 특징 선택 방법
}

model1 = RandomForestClassifier(random_state=RANDOM_STATE)

model1_rf_grid_search = GridSearchCV(estimator=model1, param_grid=param_grid, 
                           scoring='f1_weighted', cv=5, verbose=2, n_jobs=-1)

model1_rf_grid_search.fit(X_model1, y_model1)

Fitting 5 folds for each of 90 candidates, totalling 450 fits
[CV] END criterion=gini, max_depth=30, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   1.1s
[CV] END criterion=gini, max_depth=30, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   2.1s
[CV] END criterion=gini, max_depth=30, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   2.1s
[CV] END criterion=gini, max_depth=30, min_samples_leaf=1, min_samples_split=10, n_estimators=500; total time=   5.3s
[CV] END criterion=gini, max_depth=30, min_samples_leaf=1, min_samples_split=20, n_estimators=200; total time=   2.0s
[CV] END criterion=gini, max_depth=30, min_samples_leaf=1, min_samples_split=20, n_estimators=200; total time=   2.0s
[CV] END criterion=gini, max_depth=30, min_samples_leaf=1, min_samples_split=20, n_estimators=500; total time=   5.0s
[CV] END criterion=gini, max_depth=30, min_samples_leaf=2, min_samples_split=10, n_estimators=100; total time=  

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'criterion': ['gini'], 'max_depth': [30, 50, None],
                         'min_samples_leaf': [1, 2, 4, 8, 16],
                         'min_samples_split': [10, 20],
                         'n_estimators': [100, 200, 500]},
             scoring='f1_weighted', verbose=2)

In [25]:
print("Best1 parameters found: ", model1_rf_grid_search.best_params_)
print("Best1 parameters score: ", model1_rf_grid_search.best_score_)

Best1 parameters found:  {'criterion': 'gini', 'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 100}
Best1 parameters score:  0.734290392827968


In [26]:
model2 = RandomForestClassifier(random_state=RANDOM_STATE)
                                
model2_rf_grid_search = GridSearchCV(estimator=model2, param_grid=param_grid, 
                           scoring='f1_weighted', cv=5, verbose=2, n_jobs=-1)

model2_rf_grid_search.fit(X_model2, y_model2)

Fitting 5 folds for each of 90 candidates, totalling 450 fits
[CV] END criterion=gini, max_depth=50, min_samples_leaf=8, min_samples_split=20, n_estimators=200; total time=   1.7s
[CV] END criterion=gini, max_depth=50, min_samples_leaf=8, min_samples_split=20, n_estimators=500; total time=   4.2s
[CV] END criterion=gini, max_depth=50, min_samples_leaf=16, min_samples_split=10, n_estimators=100; total time=   0.8s
[CV] END criterion=gini, max_depth=50, min_samples_leaf=16, min_samples_split=10, n_estimators=200; total time=   1.5s
[CV] END criterion=gini, max_depth=50, min_samples_leaf=16, min_samples_split=10, n_estimators=200; total time=   1.5s
[CV] END criterion=gini, max_depth=50, min_samples_leaf=16, min_samples_split=10, n_estimators=500; total time=   3.7s
[CV] END criterion=gini, max_depth=50, min_samples_leaf=16, min_samples_split=20, n_estimators=100; total time=   0.8s
[CV] END criterion=gini, max_depth=50, min_samples_leaf=16, min_samples_split=20, n_estimators=100; total t

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'criterion': ['gini'], 'max_depth': [30, 50, None],
                         'min_samples_leaf': [1, 2, 4, 8, 16],
                         'min_samples_split': [10, 20],
                         'n_estimators': [100, 200, 500]},
             scoring='f1_weighted', verbose=2)

In [27]:
print("Best2 parameters found: ", model2_rf_grid_search.best_params_)
print("Best2 parameters score: ", model2_rf_grid_search.best_score_)

Best2 parameters found:  {'criterion': 'gini', 'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 500}
Best2 parameters score:  0.7671735404778526


In [28]:
model1_rf = RandomForestClassifier(**model1_rf_grid_search.best_params_, random_state=RANDOM_STATE, class_weight='balanced')
model1_rf.fit(X_model1, y_model1)
model2_rf = RandomForestClassifier(**model2_rf_grid_search.best_params_, random_state=RANDOM_STATE, class_weight='balanced')
model2_rf.fit(X_model2, y_model2)

RandomForestClassifier(class_weight='balanced', max_depth=50,
                       min_samples_split=10, n_estimators=500, random_state=42)

## XGBoost

### 모델 나누어서

In [29]:
# 데이터와 타겟 분리
X_model1 = df_model1.drop(columns='target')
y_model1 = df_model1['target']
X_model2 = df_model2.drop(columns='target')
y_model2 = df_model2['target']

y_model1_encoded = y_model1.replace({'Normal': 0, 'AbNormal': 1})
y_model2_encoded = y_model2.replace({'Normal': 0, 'AbNormal': 1})

In [30]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

# 모델 훈련
model1 = XGBClassifier(random_state=RANDOM_STATE)
model1.fit(X_model1, y_model1_encoded)

model2 = XGBClassifier(random_state=RANDOM_STATE)
model2.fit(X_model2, y_model2_encoded)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [31]:
param_grid = {
    'max_depth': [3, 5, 7],
    'eta': [0.01, 0.1, 0.3],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'reg_alpha': [0, 0.1, 1],
    'reg_lambda': [0.5, 1, 2]
}

model1 = XGBClassifier(random_state=42)

model1_xgb_grid_search = GridSearchCV(estimator=model1, param_grid=param_grid, 
                           scoring='f1_weighted', cv=5, verbose=2, n_jobs=-1)

model1_xgb_grid_search.fit(X_model1, y_model1_encoded)

Fitting 5 folds for each of 729 candidates, totalling 3645 fits
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=20, n_estimators=500; total time=   3.4s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=2, min_samples_split=10, n_estimators=100; total time=   0.7s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time=   1.4s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=2, min_samples_split=10, n_estimators=500; total time=   3.4s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=2, min_samples_split=10, n_estimators=500; total time=   3.4s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=2, min_samples_split=20, n_estimators=500; total time=   3.3s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   0.7s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=4, min_samples_split=10, n_estimators=

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=42, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.6, 0.8, 1.0],
                         'eta': [0.01, 0.1, 0.3], 'max_depth': [3, 5, 7],
                         'reg_alpha': [0, 0.1, 1], 'reg_lambda': [0.5, 1, 2],
                         'subsample': [0.6, 0.8, 1.0]},
             scoring='f1_weighted', verbose=2)

In [32]:
model2 = XGBClassifier(random_state=RANDOM_STATE)
                                
model2_xgb_grid_search = GridSearchCV(estimator=model2, param_grid=param_grid, 
                           scoring='f1_weighted', cv=5, verbose=2, n_jobs=-1)

model2_xgb_grid_search.fit(X_model2, y_model2_encoded)

Fitting 5 folds for each of 729 candidates, totalling 3645 fits
[CV] END colsample_bytree=1.0, eta=0.3, max_depth=3, reg_alpha=1, reg_lambda=2, subsample=1.0; total time=   0.4s
[CV] END colsample_bytree=1.0, eta=0.3, max_depth=3, reg_alpha=1, reg_lambda=2, subsample=1.0; total time=   0.4s
[CV] END colsample_bytree=1.0, eta=0.3, max_depth=5, reg_alpha=0, reg_lambda=0.5, subsample=0.6; total time=   0.5s
[CV] END colsample_bytree=1.0, eta=0.3, max_depth=5, reg_alpha=0, reg_lambda=0.5, subsample=0.8; total time=   0.5s
[CV] END colsample_bytree=1.0, eta=0.3, max_depth=5, reg_alpha=0, reg_lambda=0.5, subsample=1.0; total time=   0.5s
[CV] END colsample_bytree=1.0, eta=0.3, max_depth=5, reg_alpha=0, reg_lambda=0.5, subsample=1.0; total time=   0.5s
[CV] END colsample_bytree=1.0, eta=0.3, max_depth=5, reg_alpha=0, reg_lambda=1, subsample=0.6; total time=   0.6s
[CV] END colsample_bytree=1.0, eta=0.3, max_depth=5, reg_alpha=0, reg_lambda=1, subsample=0.8; total time=   0.5s
[CV] END colsamp

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=42, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.6, 0.8, 1.0],
                         'eta': [0.01, 0.1, 0.3], 'max_depth': [3, 5, 7],
                         'reg_alpha': [0, 0.1, 1], 'reg_lambda': [0.5, 1, 2],
                         'subsample': [0.6, 0.8, 1.0]},
             scoring='f1_weighted', verbose=2)

In [33]:
model1_xgb = XGBClassifier(**model1_xgb_grid_search.best_params_, random_state=RANDOM_STATE, class_weight='balanced')
model1_xgb.fit(X_model1, y_model1_encoded)
model2_xgb = XGBClassifier(**model2_xgb_grid_search.best_params_, random_state=RANDOM_STATE, class_weight='balanced')
model2_xgb.fit(X_model2, y_model2_encoded)

/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [14:58:21] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "class_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [14:58:21] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "class_weight" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              class_weight='balanced', colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.6, device=None,
              early_stopping_rounds=None, enable_categorical=False, eta=0.3,
              eval_metric=None, feature_types=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None, ...)

In [34]:
print("Best1 parameters found: ", model1_xgb_grid_search.best_params_)
print("Best1 parameters score: \n", model1_xgb_grid_search.best_score_)

print("Best2 parameters found: ", model2_xgb_grid_search.best_params_)
print("Best2 parameters score: \n", model2_xgb_grid_search.best_score_)

Best1 parameters found:  {'colsample_bytree': 1.0, 'eta': 0.3, 'max_depth': 3, 'reg_alpha': 0, 'reg_lambda': 1, 'subsample': 0.6}
Best1 parameters score: 
 0.7413950772926504
Best1 parameters found:  {'colsample_bytree': 0.6, 'eta': 0.3, 'max_depth': 5, 'reg_alpha': 0, 'reg_lambda': 0.5, 'subsample': 1.0}
Best1 parameters score: 
 0.7749594280198977


# SVC

In [35]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

model1_svm = SVC(kernel = 'rbf',random_state=RANDOM_STATE)

parameters = {'C': [0.1, 1, 10, 50, 100],
             'gamma':[0.001, 0.01, 0.1, 1]}

model1_svm_grid_search = GridSearchCV(model1_svm,
                      param_grid = parameters, cv = 5)

model1_svm_grid_search.fit(X_model1, y_model1)

[CV] END colsample_bytree=1.0, eta=0.3, max_depth=7, reg_alpha=0, reg_lambda=2, subsample=1.0; total time=   0.5s
[CV] END colsample_bytree=1.0, eta=0.3, max_depth=7, reg_alpha=0.1, reg_lambda=0.5, subsample=0.6; total time=   0.6s
[CV] END colsample_bytree=1.0, eta=0.3, max_depth=7, reg_alpha=0.1, reg_lambda=0.5, subsample=0.8; total time=   0.6s
[CV] END colsample_bytree=1.0, eta=0.3, max_depth=7, reg_alpha=0.1, reg_lambda=0.5, subsample=1.0; total time=   0.6s
[CV] END colsample_bytree=1.0, eta=0.3, max_depth=7, reg_alpha=0.1, reg_lambda=0.5, subsample=1.0; total time=   0.5s
[CV] END colsample_bytree=1.0, eta=0.3, max_depth=7, reg_alpha=0.1, reg_lambda=1, subsample=0.6; total time=   0.6s
[CV] END colsample_bytree=1.0, eta=0.3, max_depth=7, reg_alpha=0.1, reg_lambda=1, subsample=0.8; total time=   0.6s
[CV] END colsample_bytree=1.0, eta=0.3, max_depth=7, reg_alpha=0.1, reg_lambda=1, subsample=1.0; total time=   0.5s
[CV] END colsample_bytree=1.0, eta=0.3, max_depth=7, reg_alpha=0.1

GridSearchCV(cv=5, estimator=SVC(random_state=42),
             param_grid={'C': [0.1, 1, 10, 50, 100],
                         'gamma': [0.001, 0.01, 0.1, 1]})

In [36]:
result1 = pd.DataFrame(model1_svm_grid_search.cv_results_['params'])
result1['mean_test_score'] = model1_svm_grid_search.cv_results_['mean_test_score']
result1.sort_values(by='mean_test_score', ascending=False)

,C,gamma,mean_test_score
4,1.0,0.001,0.800454
6,1.0,0.100,0.798323
0,0.1,0.001,0.797612
7,1.0,1.000,0.797612
15,50.0,1.000,0.797612
11,10.0,1.000,0.797612
1,0.1,0.010,0.797612
19,100.0,1.000,0.797612
3,0.1,1.000,0.797612
2,0.1,0.100,0.797612


In [37]:
model2_svm = SVC(kernel = 'rbf',random_state=RANDOM_STATE)
model2_svm_grid_search = GridSearchCV(model2_svm,
                      param_grid = parameters, cv = 5)

model2_svm_grid_search.fit(X_model2, y_model2)

GridSearchCV(cv=5, estimator=SVC(random_state=42),
             param_grid={'C': [0.1, 1, 10, 50, 100],
                         'gamma': [0.001, 0.01, 0.1, 1]})

In [38]:
result2 = pd.DataFrame(model2_svm_grid_search.cv_results_['params'])
result2['mean_test_score'] = model2_svm_grid_search.cv_results_['mean_test_score']
result2.sort_values(by='mean_test_score', ascending=False)

,C,gamma,mean_test_score
4,1.0,0.001,0.816424
0,0.1,0.001,0.812190
5,1.0,0.010,0.808188
18,100.0,0.100,0.807484
14,50.0,0.100,0.807484
10,10.0,0.100,0.807484
6,1.0,0.100,0.805366
7,1.0,1.000,0.803954
1,0.1,0.010,0.803954
11,10.0,1.000,0.803954


In [39]:
model1_svm = SVC(**model1_svm_grid_search.best_params_, kernel = 'rbf', random_state=RANDOM_STATE)
model1_svm.fit(X_model1, y_model1)
model2_svm = SVC(**model2_svm_grid_search.best_params_, kernel = 'rbf', random_state=RANDOM_STATE)
model2_svm.fit(X_model2, y_model2)

SVC(C=1, gamma=0.001, random_state=42)

In [40]:
print("Best1 parameters found: ", model1_svm_grid_search.best_params_)
print("Best1 parameters score: \n", model1_svm_grid_search.best_params_)

print("Best2 parameters found: ", model2_svm_grid_search.best_params_)
print("Best2 parameters score: \n", model2_svm_grid_search.best_params_)

Best1 parameters found:  {'C': 1, 'gamma': 0.001}
Best1 parameters score: 
 {'C': 1, 'gamma': 0.001}
Best2 parameters found:  {'C': 1, 'gamma': 0.001}
Best2 parameters score: 
 {'C': 1, 'gamma': 0.001}


## LGBM

In [44]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV

# LightGBM 모델 정의
model1_lgbm = LGBMClassifier()
model2_lgbm = LGBMClassifier()

In [45]:
model1_lgbm_grid_search_best = {
    'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 10, 'n_estimators': 200, 'num_leaves': 50, 'subsample': 0.6
    }
model2_lgbm_grid_search_best = {
    'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': -1, 'n_estimators': 500, 'num_leaves': 50, 'subsample': 0.6}
# 모델 정의 시 random_state 설정
model1_lgbmxxxx = LGBMClassifier(
    **model1_lgbm_grid_search_best,
    random_state=RANDOM_STATE, 
    class_weight='balanced'
)

model2_lgbm = LGBMClassifier(
    **model2_lgbm_grid_search_best,
    random_state=RANDOM_STATE, 
    class_weight='balanced'
)

# fit 메서드 호출 시 random_state 인자는 필요 없음
model1_lgbm.fit(X_model1, y_model1)
model2_lgbm.fit(X_model2, y_model2)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 5612, number of negative: 1424
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002581 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3357
[LightGBM] [Info] Number of data points in the train set: 7036, number of used features: 126
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.797612 -> initscore=1.371437
[LightGBM] [Info] Start training from score 1.371437
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 3416, number of negative: 833
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000879 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise

LGBMClassifier(class_weight='balanced', colsample_bytree=0.8, n_estimators=500,
               num_leaves=50, random_state=42, subsample=0.6)

# 모델1

In [46]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

scale_pos_weight = len(y_model1[y_model1 == 'AbNormal']) / len(y_model1[y_model1 == 'Normal'])

model1_rf = RandomForestClassifier(**model1_rf_grid_search.best_params_,random_state=RANDOM_STATE,class_weight='balanced')
model1_xgb = XGBClassifier(**model1_xgb_grid_search.best_params_,random_state=RANDOM_STATE)
model1_svm = SVC(**model1_svm_grid_search.best_params_, probability=True,kernel = 'rbf', random_state=RANDOM_STATE)
model1_lgbm = LGBMClassifier(**model1_lgbm_grid_search_best,class_weight='balanced',random_state=RANDOM_STATE)

# 앙상블 모델 정의 (투표 기반)
model1_voting = VotingClassifier(
    estimators=[('rf', model1_rf), ('xgb', model1_xgb), ('svm', model1_svm), ('lgbm', model1_lgbm)],
    voting='soft'
)

# 모델 학습
model1_voting.fit(X_model1, y_model1)

/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [15:09:53] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "class_weight" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 5612, number of negative: 1424
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001948 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3357
[LightGBM] [Info] Number of data points in the train set: 7036, number of used features: 126
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000


VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(class_weight='balanced',
                                                     max_depth=30,
                                                     min_samples_split=10,
                                                     random_state=42)),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            class_weight='balanced',
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=1.0, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=Fal...
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None, ...)),
                             ('svm',
                              SVC(C=1, gamma=0.001, probability=True,
                                  random_state=42)),
                             ('lgbm',
                              LGBMClassifier(class_weight='balanced',
                                             colsample_bytree=0.8,
                                             learning_rate=0.01, max_depth=10,
                                             n_estimators=200, num_leaves=50,
                                             random_state=42, subsample=0.6))],
                 voting='soft')

# 모델2

In [47]:
scale_pos_weight = len(y_model2[y_model2 == 'AbNormal']) / len(y_model2[y_model2 == 'Normal'])
RANDOM_STATE = 42

model2_rf = RandomForestClassifier(**model2_rf_grid_search.best_params_,random_state=RANDOM_STATE,class_weight='balanced')
model2_xgb = XGBClassifier(**model2_xgb_grid_search.best_params_,random_state=RANDOM_STATE)
model2_svm = SVC(**model2_svm_grid_search.best_params_,probability=True, kernel = 'rbf', random_state=RANDOM_STATE)
model2_lgbm = LGBMClassifier(**model1_lgbm_grid_search_best,class_weight='balanced',random_state=RANDOM_STATE)

# 앙상블 모델 정의 (투표 기반)
model2_voting = VotingClassifier(
    estimators=[('rf', model2_rf), ('xgb', model2_xgb), ('svm', model2_svm), ('lgbm', model2_lgbm)],
    voting='soft'
)

# 모델 학습
model2_voting.fit(X_model2, y_model2)

/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [15:10:09] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "class_weight" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 3416, number of negative: 833
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000898 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3136
[LightGBM] [Info] Number of data points in the train set: 4249, number of used features: 121
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(class_weight='balanced',
                                                     max_depth=50,
                                                     min_samples_split=10,
                                                     n_estimators=500,
                                                     random_state=42)),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            class_weight='balanced',
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.6, device=None,
                                            early_stopping_rounds=None,
                                            enabl...
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None, ...)),
                             ('svm',
                              SVC(C=1, gamma=0.001, probability=True,
                                  random_state=42)),
                             ('lgbm',
                              LGBMClassifier(class_weight='balanced',
                                             colsample_bytree=0.8,
                                             learning_rate=0.01, max_depth=10,
                                             n_estimators=200, num_leaves=50,
                                             random_state=42, subsample=0.6))],
                 voting='soft')

# 4. 제출하기

### 테스트 데이터 예측


테스트 데이터 불러오기


In [ ]:
test_data = pd.read_csv(os.path.join(ROOT_DIR, "test.csv"))
test_data

In [ ]:
# Drop columns with more than half of the values missing
drop_cols = []
for column in test_data.columns:
    if (test_data[column].notnull().sum() // 2) < test_data[
        column
    ].isnull().sum():
        drop_cols.append(column)
test_data = test_data.drop(drop_cols, axis=1)

test_data

In [ ]:
# 각 열의 NaN 값 개수 계산
nan_counts = test_data.isnull().sum()

# NaN 값이 있는 열들만 필터링
nan_columns = nan_counts[nan_counts > 0]

nan_columns

In [ ]:
# 각 열의 NaN 값이 있는 행을 표시
nan_dam = test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam'].isnull()
nan_fill1 = test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1'].isnull()
nan_fill2 = test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'].isnull()

# 세 열의 NaN 값이 모두 같은 행에서 발생했는지 확인
nan_same_rows = nan_dam & nan_fill1 & nan_fill2

# NaN 값이 동일한 행의 개수 확인
num_same_nan_rows = nan_same_rows.sum()

# 결과 출력
print(f"세 열에서 NaN 값이 동일한 행의 개수: {num_same_nan_rows}")
print(f"세 열에서 NaN 값이 동일한 행:\n{test_data[nan_same_rows]}")

In [ ]:
# 삭제할 열 리스트
columns_to_drop = [
        'Model.Suffix_Fill1', 'Model.Suffix_Fill2', 'Model.Suffix_AutoClave', 
        'Workorder_Fill1', 'Workorder_Fill2', 'Workorder_AutoClave'
]

# 열 삭제
test_data = test_data.drop(columns=columns_to_drop)

# 열 이름 변경
test_data = test_data.rename(columns={
    'Model.Suffix_Dam': 'Model.Suffix',
    'Workorder_Dam': 'Workorder'
})

test_data

In [ ]:
# 모든 행이 동일한 값을 가지는 열을 찾고, 그 값을 함께 저장
constant_columns = {col: test_data[col].iloc[0] for col in test_data.columns if test_data[col].nunique() == 1}

# 데이터 프레임에서 해당 열 삭제
test_data = test_data.drop(columns=constant_columns.keys())

# 삭제된 열의 개수 출력
print(f"삭제된 열의 개수: {len(constant_columns)}")

# 삭제된 열과 그 값을 한 줄에 하나씩 출력
print("삭제된 열과 값:")
for col, value in constant_columns.items():
    print(f"{col}: {value}")

test_data

In [ ]:
# 변환할 열들의 리스트
columns_to_convert = [
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2']

# 각 열을 float로 변환
for col in columns_to_convert:
    test_data[col] = pd.to_numeric(test_data[col], errors='coerce')

test_data[columns_to_convert].dtypes

In [ ]:
# # 제거할 열 리스트
columns_to_drop = ['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam', 
                   'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1', 
                   'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2']
#   'WorkMode Collect Result_Dam', 'WorkMode Collect Result_Fill1', 'WorkMode Collect Result_Fill2'
# # 지정한 열 제거
test_data = test_data.drop(columns=columns_to_drop)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline

# 범주형 열을 선택하여 인코딩 수행
categorical_features = ['Equipment_Dam', 'Equipment_Fill1', 'Equipment_Fill2', 'Chamber Temp. Judge Value_AutoClave', 'Model.Suffix', 'Workorder']

# OrdinalEncoder 인스턴스 생성
encoder = OrdinalEncoder()

# 범주형 열에 대해서만 인코딩 수행
test_data[categorical_features] = encoder.fit_transform(test_data[categorical_features])

# 인코딩된 데이터프레임 확인
test_data

In [ ]:
categorical_features = ['Equipment_Dam', 'Equipment_Fill1', 'Equipment_Fill2']

# 세 가지 조건: 열 간 값이 다르면 불량으로 분류
condition_receip = (
    (test_data['Receip No Collect Result_Dam'] != test_data['Receip No Collect Result_Fill1']) |
    (test_data['Receip No Collect Result_Dam'] != test_data['Receip No Collect Result_Fill2']) |
    (test_data['Receip No Collect Result_Fill1'] != test_data['Receip No Collect Result_Fill2'])
)

condition_production_qty = (
    (test_data['Production Qty Collect Result_Dam'] != test_data['Production Qty Collect Result_Fill1']) |
    (test_data['Production Qty Collect Result_Dam'] != test_data['Production Qty Collect Result_Fill2']) |
    (test_data['Production Qty Collect Result_Fill1'] != test_data['Production Qty Collect Result_Fill2'])
)

test_equip1 = test_data[categorical_features].eq(0).all(axis=1)
test_equip2 = test_data[categorical_features].eq(1).all(axis=1)
condition_equip_different = ~test_equip1 & ~test_equip2 

# 조건들을 묶어서 필터링
test_condition = ( condition_receip | condition_production_qty | condition_equip_different )

test_abnormal = test_data[test_condition]
# abnormal로 제외하고 남은 데이터

test_filtered = test_data[~test_condition]

# 장비가 0 또는 1의 값으로 나누기 (label encoding으로 Equipment #1: 0 , Equipment #2: 1)
test_model1 = test_filtered[test_filtered[categorical_features].eq(0).all(axis=1)]
test_model2 = test_filtered[test_filtered[categorical_features].eq(1).all(axis=1)]

test_model1_for_prediction = test_model1.drop(columns=['Set ID'])
test_model2_for_prediction = test_model2.drop(columns=['Set ID'])


test_model1_predictions = model1_voting.predict(test_model1_for_prediction)
test_model2_predictions = model2_voting.predict(test_model2_for_prediction)

# 예측을 위한 데이터 인덱스
test_model1_index = test_model1.index
test_model2_index = test_model2.index

# 검증 데이터의 전체 예측을 위한 결과 결합
final_predictions = []

for idx in test_data.index:
    if idx in test_model1_index:
        final_predictions.append(test_model1_predictions[test_model1_index.get_loc(idx)])
    elif idx in test_model2_index:
        final_predictions.append(test_model2_predictions[test_model2_index.get_loc(idx)])
    else:
        final_predictions.append('AbNormal')

In [ ]:
for i in range(len(final_predictions)):
    if final_predictions[i] == 0:
        final_predictions[i] = "Normal"
    elif final_predictions[i] == 1:
        final_predictions[i] = "AbNormal"
final_predictions

### 제출 파일 작성


In [ ]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("./submission.csv")
df_sub["target"] = final_predictions

# 제출 파일 저장
df_sub.to_csv("./submission.csv", index=False)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**
